In [ ]:
import random
import re
import ipywidgets as widgets
from IPython.display import display

In [ ]:
from dicesapi import DicesAPI
api = DicesAPI(dices_api='http://localhost:8000/api')
# this is just to provide progress bars in Jupyter
from dicesapi.jupyter import NotebookPBar
api._ProgressClass = NotebookPBar

# Download some speeches

In [ ]:
speeches = api.getSpeeches(work_title='Iliad', progress=True) + \
            api.getSpeeches(work_title='Odyssey', progress=True)

In [ ]:
sorted(speeches._things, key=lambda s: s.part)

# Download the text of the speeches

In [ ]:
passages = [None] * len(speeches)

# create a progress bar
pbar = widgets.IntProgress(
    value = 0,
    min = 0,
    max = len(speeches),
    bar_style='info',
    orientation='horizontal'
)
pbar_label = widgets.Label(value = f'Downloading {pbar.value}/{len(speeches)}')
display(widgets.HBox([pbar, pbar_label]))

# download text of speeches
for i, s in enumerate(speeches):
    cts_passage = s.getCTS()
    text = cts_passage.text
    passages[i] = text
    pbar.value = i
    pbar_label.value = f'Downloading {i+1}/{len(speeches)}'

# Part of speech tagging

In [ ]:
from cltk.tag.pos import POSTag
tagger = {
    'greek': POSTag('greek').tag_ngram_123_backoff,    
    'latin': POSTag('latin').tag_ngram_123_backoff,
}

In [ ]:
POS_tags = [None] * len(passages)

# create a progress bar
pbar = widgets.IntProgress(
    value = 0,
    min = 0,
    max = len(speeches),
    bar_style='info',
    orientation='horizontal'
)
pbar_label = widgets.Label(value = f'POS tagging {pbar.value}/{len(passages)}')
display(widgets.HBox([pbar, pbar_label]))

# download text of speeches
for i, p in enumerate(passages):
    lang = speeches[i].work.lang
    tagged = tagger[lang](p.lower())
    POS_tags[i] = tagged
    pbar.value = i
    pbar_label.value = f'POS tagging {i+1}/{len(passages)}'